# End of term project for Deep Learning course, SS 2020 @ University of Wrocław
Authors:
* Piotr Gdowski
* Michał Martusewicz


# Main goal
Extend the handwritten digit-recognizing network to allow verifying sudoku's solution correctness

**Before start: runtime -> change runtime type -> GPU**

In [ ]:
%pylab inline
%matplotlib inline
%load_ext autoreload
%autoreload 2

from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from tqdm.notebook import tqdm


try:
    from src.net import get_test_and_train_dataloader, Net, train, test, FinalNet, get_dataloader
    from src.preprocessing import deskew_verify, Deskewing, split_into_cells
    from src.sudoku_app import get_predictions, check
    from src.util import get_gray_images, get_pics_path, present_dataset, get_pure_data
    
except ModuleNotFoundError:
    import httpimport
    with httpimport.github_repo(
            'iCarrrot', 
            'nn_sudoku_project',
            module='src',
            branch='master'
    ):
        from src.net import get_test_and_train_dataloader, Net, train, test, FinalNet, get_dataloader
        from src.preprocessing import deskew_verify, Deskewing, split_into_cells
        from src.sudoku_app import get_predictions, check
        from src.util import get_gray_images, get_pics_path, present_dataset, get_pure_data


GridInstance = namedtuple('GridInstace', 'img digits')

# Dataset

We've collected and labeled a dataset consisting of 101 sudoku grids, which we splitted into three parts after preprocessing:
* 10 grids will be our test dataset on which we'll show how the full checker works
* ca. 5000 digits will be put to the model as a training data
* the rest will serve as verification dataset

In [ ]:
# Load dataset
pics_path = get_pics_path()
gray_images, labels = get_gray_images(pics_path)
gray_images.shape

In [ ]:
deskewed_imgs = []
desk_img_labels = []
plt.figure(figsize=(20, 20))
for i in tqdm(range(len(gray_images))):
    deskewed = Deskewing(gray_images[i])._deskew()
    plt.subplot(11, 10, i + 1)
    if i == 1 or not deskew_verify(deskewed):
        plt.imshow(deskewed)
    else:
        plt.imshow(deskewed, cmap='gray')
        deskewed_imgs.append(deskewed)
        desk_img_labels.append(labels[i])

## Dataset split

In [ ]:
testset_ids = np.array([9,12,13,22,24,27,32,35,37,50])

testset, trainset = [], []
for i in range(len(desk_img_labels)):
    if i in testset_ids:
        testset.append(GridInstance(img=deskewed_imgs[i], digits=np.asarray(desk_img_labels[i])))
    else:
        trainset.append(GridInstance(img=deskewed_imgs[i], digits=np.asarray(desk_img_labels[i])))

print(sorted(testset_ids))

digits_labels = np.hstack([g.digits for g in trainset])
digits = np.vstack([split_into_cells(g.img) for g in trainset])
digits.shape

## Dataloaders and parameters

In [ ]:
batch_size_train = 64
batch_size_test = 32

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('files/', train=True, download=True,
        transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])), 
    batch_size=batch_size_train, 
    shuffle=True)

grid_test_loader, grid_train_loader = get_test_and_train_dataloader(
    digits, 
    digits_labels, 
    batch_size_test = batch_size_test, 
    device='cuda')


## MNIST Dataset - sample digits and T-SNE decomposition

In [ ]:
present_dataset(train_loader)

## Digits from sudoku grids - sample digits and T-SNE decomposition

In [ ]:
present_dataset(grid_train_loader)

### PCA decomposition on sudoku data

In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform(digits.reshape(-1, 28*28))
X.shape
plt.figure(figsize=(15,15))
plt.scatter(X[:,0], X[:,1], c=digits_labels)
plt.colorbar()


# Classification

In this section we'll compare several approaches on classifying sudoku grids' digits.

## KNN

### 1: train on MNIST, test on sudoku_test

In [ ]:
X, y = get_pure_data(train_loader)
X_test, y_test = get_pure_data(grid_test_loader)

for n_components in [10, 20, 30, 35, 40, 45, 50, 70, 100, 200]:
    pca = PCA(n_components=n_components)
    pca.fit(X)
    
    pca_train = pca.transform(X)
    pca_test = pca.transform(X_test)
    
    clf = KNeighborsClassifier()
    clf.fit(pca_train, y)

    print(f"{n_components} components: {clf.score(pca_test, y_test)}")

### 2: train on PCA(sudoku_train), test on PCA(sudoku_test)

In [ ]:
X, y = get_pure_data(grid_train_loader)
X_test, y_test = get_pure_data(grid_test_loader)

for n_components in [10, 20, 30, 35, 40, 45, 50, 70, 100, 200]:
    pca = PCA(n_components=n_components)
    pca.fit(X)
    
    pca_train = pca.transform(X)
    pca_test = pca.transform(X_test)
    
    clf = KNeighborsClassifier()
    clf.fit(pca_train, y)

    print(f"{n_components} components: {clf.score(pca_test, y_test)}")

## NN

### Simple NN on different datasets

In [ ]:
model = Net()
model.to('cuda')
train(model=model, device='cuda', train_loader=train_loader, title="Train on mnist dataset", optimizer=torch.optim.Adam(model.parameters()))
prediction = test(model=model, device='cuda', test_loader=grid_test_loader)

In [ ]:
model = Net()
model.to('cuda')
train(model=model, device='cuda', train_loader=grid_train_loader, title="Train on own dataset", optimizer=torch.optim.Adam(model.parameters()))
prediction = test(model=model, device='cuda', test_loader=grid_test_loader)

### Final Net

In [ ]:
model = FinalNet()
model.to('cuda')
train(model=model, device='cuda', train_loader=grid_train_loader, epoch=60, title="Train on own dataset", optimizer=torch.optim.Adam(model.parameters()))
prediction = test(model=model, device='cuda', test_loader=grid_test_loader)
torch.save(model.state_dict(), 'drive/My Drive/dataset/final_net.nn')

### Reload final net

In [ ]:
restored_model = FinalNet()
restored_model.load_state_dict(torch.load('drive/My Drive/dataset/final_net.nn'))
restored_model.cuda()
prediction = test(model=restored_model, device='cuda', test_loader=grid_test_loader)

# Real life application

Given a sudoku grid, find errors or say it's fine

In [ ]:
img, y = testset[4]
grid_cells = split_into_cells(img)
nonempty_cells = []
for idx, cell in enumerate(grid_cells):
    nonempty_cells.append(idx)

grid_dataloader = get_dataloader(grid_cells[nonempty_cells], y[nonempty_cells])

predicted, ppbs = get_predictions(restored_model, 'cuda', grid_dataloader)


In [ ]:
for gi in testset:
    check(gi, restored_model, 0.95)

# What have we learnt
* Colab sucks
* MNIST didn’t help
* Graphic card has limited memory

# Further work

* Collect more labeled data -> different handwriting styles
* Improve data preprocessing
* Better net architecture
* Release as mobile app
* Different approach?
    * Train neural net with the whole sudoku diagram